In [35]:
import pandas as pd
import os
#import nltk
import string
#from nltk.corpus import stopwords        
#import matplotlib.pyplot as plt
import numpy as np
import datetime

In [36]:
os.chdir("F:\\Projects\\FlightTicketPrice\\Flight_Ticket_Participant_Datasets")
os.getcwd()

'F:\\Projects\\FlightTicketPrice\\Flight_Ticket_Participant_Datasets'

In [39]:
traindf = pd.read_excel("Data_Train.xlsx")
traincount = len(traindf)
print(traincount)

10683


In [40]:
testdf = pd.read_excel("Test_set.xlsx")
testdf.head()
testcount = len(testdf)
print(testcount)

2671


In [41]:
traindf.shape, testdf.shape

((10683, 11), (2671, 10))

In [42]:
testdf["Price"] = 0 

In [43]:
trainMerged = traindf
trainMerged = trainMerged.append(testdf,ignore_index=True)
trainMerged.shape

(13354, 11)

In [44]:
trainMerged.to_csv("MergedFile.csv")

In [45]:
dfMerged = pd.read_csv("MergedFileWithComma.csv")
dfMerged.shape

(13354, 16)

In [46]:
dfMerged.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time',
       'Arrival_Time', 'Duration', 'Total_Stops', 'Additional_Info', 'Price',
       'RT0', 'RT1', 'RT2', 'RT3', 'RT4', 'RT5'],
      dtype='object')

In [47]:
## Functions 
## Concert time in text (xxh xxmin) to minutes
def getTimeInMin(text):
    if "h" in text:
#        print("h available")
        hour, minute = text.split("h")
        if "m" in text:
#            print("min available")
            minutes = (60 * int(hour)) + int(minute.strip("m"))
        else:        
            hour, minute = text.split("h")
            minutes = 60 * int(hour)
    else:
            minutes = int(text.strip("m"))
    return minutes

## Divide day into different parts. Take time of Arrival or Departure and return part of the day
def get_part_of_day(text):
    text = text[:5]
#    print(text)
    hour, minute = text.split(":")
    hour = int(hour)
    return (
        "EM" if 4 <= hour <= 5
        else
        "MO" if 6 <= hour <= 9
        else
        "LM" if 10 <= hour <= 11
        else
        "AN" if 12 <= hour <= 16
        else
        "EV" if 17 <= hour <= 19
        else
        "NI" if 20 <= hour <= 22
        else
        "LN"
    )

## Take date in text format and return day of the week
def getDayOfWeek(text):
    from datetime import datetime
#    day = datetime.datetime.strptime('January 11, 2010', '%B %d, %Y').strftime('%A')
#    day = datetime.datetime.strptime("02/03/2017", '%d/%m/%Y').strftime('%A')
    day = datetime.strptime(text, '%d-%m-%Y').strftime('%A')
    return day

## Take date in text format as input and return day of the year
def getDayOfYear(text):
    from datetime import datetime
    adate = datetime.strptime(text,"%d-%m-%Y")
    day_of_year = adate.timetuple().tm_yday
    return day_of_year

def getTime(text):
    from datetime import datetime
    if (len(text) == 5):
        sTime = text
    else:
        sTime = text[-5:]
    return sTime 

In [48]:
dfMerged['RT0'] = dfMerged['RT0'].str.rstrip()
dfMerged['RT0'] = dfMerged['RT0'] + "0"
dfMerged['RT1'] = dfMerged['RT1'].str.rstrip()
dfMerged['RT1'] = dfMerged['RT1'] + "1"
dfMerged['RT2'] = dfMerged['RT2'].str.rstrip()
dfMerged['RT2'] = dfMerged['RT2'] + "2"
dfMerged['RT3'] = dfMerged['RT3'].str.rstrip()
dfMerged['RT3'] = dfMerged['RT3'] + "3"
dfMerged['RT4'] = dfMerged['RT4'].str.rstrip()
dfMerged['RT4'] = dfMerged['RT4'] + "4"
dfMerged['RT5'] = dfMerged['RT5'].str.rstrip()
dfMerged['RT5'] = dfMerged['RT5'] + "5"

In [17]:
#print(dfMerged['RT0'].head())
#print(dfMerged['RT1'].head())
#print(dfMerged['RT2'].head())
#print(dfMerged['RT3'].head())
#print(dfMerged['RT4'].unique())
#print(dfMerged['RT5'].unique())

[nan ' COK4' ' BLR4' ' DEL4' ' HYD4' ' VGA4']


In [49]:
## Arival Time - Part of the Day 

dfMerged['ArrSession'] = ""
dfMerged['Arrival_Time'] = dfMerged.apply(lambda row: getTime(row['Arrival_Time']), axis=1)
#dfMerged['Arrival_Time'].head()
dfMerged['ArrSession'] = dfMerged.apply(lambda row: get_part_of_day(row['Arrival_Time']), axis=1)
dfMerged['ArrSession'] = 'Arr' + dfMerged['ArrSession']
dfMerged['ArrSession'].head()

0    ArrLN
1    ArrAN
2    ArrEM
3    ArrLN
4    ArrNI
Name: ArrSession, dtype: object

In [50]:
## Departure Time - Part of the Day 

dfMerged['DepSession'] = ""
dfMerged['Dep_Time'] = dfMerged.apply(lambda row: getTime(row['Dep_Time']), axis=1)
#dfMerged['Arrival_Time'].head()
dfMerged['DepSession'] = dfMerged.apply(lambda row: get_part_of_day(row['Dep_Time']), axis=1)
dfMerged['DepSession'] = 'Dep' + dfMerged['DepSession']
dfMerged['DepSession'].head()

0    DepNI
1    DepEM
2    DepMO
3    DepEV
4    DepAN
Name: DepSession, dtype: object

In [52]:
## Changing Duration from text to minutes
dfMerged['DurMin'] = ""
dfMerged['DurMin'] = dfMerged.apply(lambda row: getTimeInMin(row['Duration']), axis=1)
#dfMerged['DurMin'] = dfMerged.apply
dfMerged['DurMin'].head()

0     170
1     445
2    1140
3     325
4     285
Name: DurMin, dtype: int64

In [54]:
## Extract the Day of the week from the date field. 

from datetime import datetime

dfMerged['DayOfWeek'] = ""
dfMerged['DayOfWeek'] = dfMerged.apply(lambda row: getDayOfWeek(row['Date_of_Journey']), axis=1)

In [55]:
## Pass the Date in text format and get the Day of the Year 
dfMerged['DayOfYear'] = ""
dfMerged['DayOfYear'] = dfMerged.apply(lambda row: getDayOfYear(row['Date_of_Journey']), axis=1)

In [56]:
dfMerged['DayOfWeek'].head()

0       Sunday
1    Wednesday
2       Sunday
3       Sunday
4       Friday
Name: DayOfWeek, dtype: object

In [57]:
dfMerged['DayOfYear'].head()

0     83
1    121
2    160
3    132
4     60
Name: DayOfYear, dtype: int64

In [60]:
#trainMerged['DayOfWeek'] = trainMerged['DayOfWeek'].astype(str)
#trainMerged['DayOfWeek'] = 'WDay' + trainMerged['DayOfWeek']

In [58]:
dfMerged['DayOfYear'] = dfMerged['DayOfYear'].astype(str)
dfMerged['DayOfYear'] = 'YDay' + dfMerged['DayOfYear']

### Convert New Delhi to Delhi as both are same. 

In [60]:
%%timeit

dfMerged.loc[dfMerged['Destination'] == 'New Delhi', 'Destination'] = 'Delhi'

3.31 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
print(dfMerged['Source'].unique())
print(dfMerged['Destination'].unique())

['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
['Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Hyderabad']


In [62]:
dfMerged['Source'] = "Src" + trainMerged['Source']
dfMerged['Destination'] = "Dest" + trainMerged['Destination'] 

In [63]:
Airline = pd.get_dummies(dfMerged['Airline'], drop_first=False)
DayOfWeek = pd.get_dummies(dfMerged['DayOfWeek'], drop_first=False)
DayOfYear = pd.get_dummies(dfMerged['DayOfYear'], drop_first=False)
#DurMin = pd.get_dummies(traindf['DurMin'], drop_first=False)
DepSession = pd.get_dummies(dfMerged['DepSession'], drop_first=False)
ArrSession = pd.get_dummies(dfMerged['ArrSession'], drop_first=False)
TotalStops = pd.get_dummies(dfMerged['Total_Stops'], drop_first=False)
#Additional_Info = pd.get_dummies(trainMerged['Additional_Info'], drop_first=False)
RT0 = pd.get_dummies(dfMerged['RT0'], drop_first=False)
RT1 = pd.get_dummies(dfMerged['RT1'], drop_first=False)
RT2 = pd.get_dummies(dfMerged['RT2'], drop_first=False)
RT3 = pd.get_dummies(dfMerged['RT3'], drop_first=False)
RT4 = pd.get_dummies(dfMerged['RT4'], drop_first=False)
RT5 = pd.get_dummies(dfMerged['RT5'], drop_first=False)

In [64]:
dfMerged = pd.concat([dfMerged,Airline, DayOfWeek, DayOfYear, DepSession,ArrSession, TotalStops, RT0, RT1, RT2, RT3, RT4, RT5],1)

In [65]:
dfMerged.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time',
       'Arrival_Time', 'Duration', 'Total_Stops', 'Additional_Info', 'Price',
       ...
       ' GWL3', ' HYD3', ' NAG3', ' TRV3', ' BLR4', ' COK4', ' DEL4', ' HYD4',
       ' VGA4', ' DEL5'],
      dtype='object', length=188)

In [66]:
dropcolumn = ['Airline','Source','Destination','Total_Stops','Additional_Info','DepSession','ArrSession',  'Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'DayOfWeek', 'DayOfYear', 'RT0', 'RT1', 'RT2', 'RT3', 'RT4', 'RT5']
dfMerged.drop(dropcolumn, axis=1, inplace=True)

In [114]:
dfMerged.columns

Index(['Price', 'Air Asia', 'Air India', 'GoAir', 'IndiGo', 'Jet Airways',
       'Jet Airways Business', 'Multiple carriers',
       'Multiple carriers Premium economy', 'SpiceJet',
       ...
       ' HYD', ' HYD ', ' NAG ', ' TRV ', ' BLR', ' COK', ' DEL', ' HYD',
       ' VGA ', ' DEL'],
      dtype='object', length=178)

In [67]:
dfMerged['DurMin'].head()

0     170
1     445
2    1140
3     325
4     285
Name: DurMin, dtype: int64

In [69]:
# Scale numerical columns. 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
#dfCustChurn[['tenure', 'MonthlyCharges']] = scaler.fit_transform(dfCustChurn[['tenure', 'MonthlyCharges']])
dfMerged[['DurMin']] = scaler.fit_transform(dfMerged[['DurMin']])

C:\Users\srinivas\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [70]:
dfMerged['DurMin'].head()

0    0.057793
1    0.154116
2    0.397548
3    0.112084
4    0.098074
Name: DurMin, dtype: float64

In [71]:
dfMerged.to_csv("PreProcessed.csv")

In [72]:
dfMerged.columns

Index(['Price', 'DurMin', 'Air Asia', 'Air India', 'GoAir', 'IndiGo',
       'Jet Airways', 'Jet Airways Business', 'Multiple carriers',
       'Multiple carriers Premium economy',
       ...
       ' GWL3', ' HYD3', ' NAG3', ' TRV3', ' BLR4', ' COK4', ' DEL4', ' HYD4',
       ' VGA4', ' DEL5'],
      dtype='object', length=169)

In [81]:
dfTest = dfMerged[traincount:len(dfMerged)+1]
dfTrain = dfMerged[:traincount]

In [82]:
len(dfTrain), len(dfTest)

(10683, 2671)

In [83]:
Y = dfTrain[['Price']]
X = dfTrain
X.drop('Price', axis=1, inplace=True)
X.columns

C:\Users\srinivas\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Index(['DurMin', 'Air Asia', 'Air India', 'GoAir', 'IndiGo', 'Jet Airways',
       'Jet Airways Business', 'Multiple carriers',
       'Multiple carriers Premium economy', 'SpiceJet',
       ...
       ' GWL3', ' HYD3', ' NAG3', ' TRV3', ' BLR4', ' COK4', ' DEL4', ' HYD4',
       ' VGA4', ' DEL5'],
      dtype='object', length=168)

In [94]:
Y.columns

Index(['Price'], dtype='object')

In [96]:
len(X), len(Y)

(10683, 10683)

In [186]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size= 0.25,random_state=1)

In [187]:
len(X_train), len(X_test), len(Y_train), len(Y_test)

(8012, 2671, 8012, 2671)

In [188]:
X_train.head()

,DurMin,Air Asia,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,...,GWL3,HYD3,NAG3,TRV3,BLR4,COK4,DEL4,HYD4,VGA4,DEL5
4035,0.029772,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7865,0.092820,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1362,0.619965,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4236,0.437828,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4636,0.392294,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
Y_train.head()

,Price
4035,2754
7865,4812
1362,15129
4236,15532
4636,13029


In [190]:
X_test.head()

,DurMin,Air Asia,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,...,GWL3,HYD3,NAG3,TRV3,BLR4,COK4,DEL4,HYD4,VGA4,DEL5
7931,0.028021,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5468,0.059545,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8618,0.253940,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3836,0.211909,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7771,0.441331,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
Y_test.head()

,Price
7931,2753
5468,6309
8618,13731
3836,13555
7771,10844


In [192]:
from sklearn.ensemble import RandomForestRegressor

model1 = RandomForestRegressor(n_estimators=500)
model1.fit(X_train,Y_train.values.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [193]:
import pickle
filename = 'modelRandomForest.sav'
pickle.dump(model1, open(filename, 'wb'))

In [194]:
X_train.shape, Y_train.shape

((8012, 168), (8012, 1))

In [195]:
X_test.shape, Y_test.shape

((2671, 168), (2671, 1))

In [196]:
filename = "modelRandomForest.sav"
loaded_model = pickle.load(open(filename, 'rb'))
dfPredict = pd.DataFrame(loaded_model.predict(X_test))

In [197]:
dfPredict.head()

,0
0,2037.862838
1,6098.096000
2,13028.853733
3,11381.512500
4,11120.950233


In [198]:
dfPredict.rename(columns = {list(dfPredict)[0]:'PredictedPrice'}, inplace=True)

In [201]:
dfPredict.shape

(2671, 1)

In [200]:
Y_test.head()

,Price
7931,2753
5468,6309
8618,13731
3836,13555
7771,10844


In [202]:
X_test = pd.concat([X_test,Y_test,dfPredict], axis=1)
X_test.columns

Index(['DurMin', 'Air Asia', 'Air India', 'GoAir', 'IndiGo', 'Jet Airways',
       'Jet Airways Business', 'Multiple carriers',
       'Multiple carriers Premium economy', 'SpiceJet',
       ...
       ' NAG3', ' TRV3', ' BLR4', ' COK4', ' DEL4', ' HYD4', ' VGA4', ' DEL5',
       'Price', 'PredictedPrice'],
      dtype='object', length=170)

In [203]:
X_test.head()

,DurMin,Air Asia,Air India,GoAir,IndiGo,Jet Airways,Jet Airways Business,Multiple carriers,Multiple carriers Premium economy,SpiceJet,...,NAG3,TRV3,BLR4,COK4,DEL4,HYD4,VGA4,DEL5,Price,PredictedPrice
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2037.862838
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6098.096000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13028.853733
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11381.512500
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11120.950233


In [176]:
X_test = pd.concat([X_test,dfPredict], axis=1)

In [177]:
X_test[['Price' ,'PredictedPrice']].head()

,Price,PredictedPrice
0,NaN,2038.572533
1,NaN,6097.176676
2,NaN,13003.430933
3,NaN,11416.823900
4,NaN,11085.643967


In [108]:
dfTest.drop('Price', axis=1, inplace=True)

pandas.core.frame.DataFrame

In [109]:
predictdf

,0
0,11671.590067
1,4490.033000
2,13598.899933
3,12425.606400
4,4063.745800
5,10346.790167
6,11050.732000
7,4775.236000
8,4351.160200
9,13064.422267
